A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
LIVE EVENT DATE:  June 22, 2023
LIVE EVENT TIME:  10:00 AM EDT
Use the link below to enter the event up to 15 minutes before the start.

EVENT LINK:  https://event.on24.com/wcc/r/4260101/F8F2647006EDBE0A5632E6617E76E912?mode=login&email=sungwoo77.hong@samsung.com
EMAIL:  sungwoo77.hong@samsung.com

In [ ]:
https://vimeo.com/839749267?mkt_tok=Nzg2LVpLVC0wNjQAAAGMmV74AV4JQUw05my6iXG-OurZBor4oTo--v84QvljrL9kHdKutE7FYu5Zkamp-2MyPWdf56ih_dWSsYzFq0VIPUeJxpG5z59Xzmu7L0J9sQjj

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []

    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)

            result_dict[code] = [company_name, per, debt_ratio]

        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)

    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)

    # transpose DataFrame
    result_df = result_df.transpose()

    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']

    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
